<a href="https://colab.research.google.com/github/manimoh/bahama_nma_project/blob/master/Working_example_baseline_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#@title Data retrieval
import os, requests

fname = []
for j in range(3):
  fname.append('steinmetz_part%d.npz'%j)
url = ["https://osf.io/agvxh/download"]
url.append("https://osf.io/uv3mw/download")
url.append("https://osf.io/ehmw2/download")

for j in range(len(url)):
  if not os.path.isfile(fname[j]):
    try:
      r = requests.get(url[j])
    except requests.ConnectionError:
      print("!!! Failed to download data !!!")
    else:
      if r.status_code != requests.codes.ok:
        print("!!! Failed to download data !!!")
      else:
        with open(fname[j], "wb") as fid:
          fid.write(r.content)

In [4]:
#@title Data loading
import numpy as np
import copy

alldat = np.array([])
for j in range(len(fname)):
    alldat = np.hstack((alldat, np.load('steinmetz_part%d.npz'%j, allow_pickle=True)['dat']))

# select just one of the recordings here. 11 is nice because it has some neurons in vis ctx. 
dat = alldat[11]
print(dat.keys())

# Making a backup so that we don't have to load the data back up again
alldat_backup = copy.copy(alldat)

dict_keys(['spks', 'wheel', 'pupil', 'response', 'response_time', 'bin_size', 'stim_onset', 'contrast_right', 'contrast_left', 'brain_area', 'feedback_time', 'feedback_type', 'gocue', 'mouse_name', 'date_exp', 'trough_to_peak', 'active_trials', 'contrast_left_passive', 'contrast_right_passive', 'spks_passive', 'pupil_passive', 'wheel_passive', 'prev_reward', 'ccf', 'ccf_axes', 'cellid_orig', 'reaction_time', 'face', 'face_passive', 'licks', 'licks_passive'])


In [5]:
#@title Import statements (Add more as needed)
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import random
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [6]:
#@title Helper functions
def rebin_st(st, bin_size):
    '''
    Method to rebin a 2D spike train by summing the spikes from nearby bins

    Inputs:
    st: original spike_train of n neurons * t timebins
    bin_size: Desired bin size (Must be a factor of 't')

    Returns:
    rst: spike_train of n neurons * bin_size timebins
    '''
    rst = np.zeros((st.shape[0], st.shape[1]//bin_size))
    for i in range(rst.shape[1]):
        rst[:,i] = np.sum(st[:,i*bin_size:(i+1)*bin_size], axis=1)
    return rst


def make_features_for_this_region (base_features):
    '''
    Method to make rows of a design_matrix with intra region interaction terms

    Inputs:
    base_features:
    Top n principal components of this region

    Returns:
    A 1D numpy array with all the base features and second order interaction
    terms 
    '''
    features = []
    # First add the base terms
    for f in (base_features):
        features.append(f)
    # Next add the interaction terms
    for i in range(0,len(base_features)):
        features.append(base_features[i]**2)
        for j in range(i+1, len(base_features)):
            features.append(base_features[i]*base_features[j])
    return np.array(features)


def make_features_for_these_regions (base_features_1, base_features_2):
    '''
    Method to create cross-interactions terms from two regions

    Inputs:
    base_features_1 :
    Top n principal components of this region
    base_features_2 :
    Top n principal components of this region

    Returns:
    A 1D numpy array with all the inter-region pairwise interaction terms
    '''
    features = []
    #interactive_terms        
    for i in range(0,len(base_features_1)):
        for j in range(0, len(base_features_2)):
            features.append(base_features_1[i]*base_features_2[j])
    return np.array(features)

def make_primary_featues (top_n, data):
    '''
    Method to return a dictionary of primary features based on the top_n PCA
    components to be selected from the data

    Inputs:
    top_n :
    Top n principal components to be selected
    data :
    Dictionary with brain region as key and the corresponding data as value

    Returns:
    primary_features_dict
   Dictionary with brain region as key and the top_n features as value
    '''
    primary_features_dict = {}
    for key in data:
        this_data = data[key]
        pca = PCA(n_components=top_n)
        pca.fit(this_key)
        this_pca = pca.components_
        this_features = (this_pca @ this_data.T).T
        primaary_features_dict[key] = this_features
    return primary_features_dict


In [10]:
#@title Preprocessing Data
# Combining areas based on our 'expertise' (LOL)
combined_areas = {
    'MoCo': ['MOs', 'MOp'],
    'VisualAreas': ['VISa', 'VISam', 'VISI', 'VISp', 'VISpm', 'VISrl'],
    'Hippocampus' : ['CA1', 'CA2', 'CA3', 'DG'],
    'SomNuc' : ['VPL', 'VPM']
}
# print(combined_areas.keys())

# Creating a new key called 'new area' in alldat to add new_labels
for dat in alldat:
    new_labels = []
    old_labels = dat['brain_area']
    for label in old_labels:
        found = False
        for key in combined_areas:
            if label in combined_areas[key]:
                new_labels.append(key)
                found = True
                break
        # if not present in the combined_areas dict
        if not found:
            new_labels.append(label)
    dat['new_area'] = np.array(new_labels)

sessions = [np.unique(dat['new_area']) for dat in alldat]
chosen_ones = ['ACA', 'Hippocampus', 'MRN', 'MoCo', 'PL', 'SUB', 'VisualAreas']
overlap  = pd.DataFrame(index=chosen_ones, columns=chosen_ones)
for ses in sessions:
    for i in range(len(ses)-1):
        for j in range(i+1,len(ses)):
            if ses[i] in chosen_ones and ses[j] in chosen_ones:
                if overlap.loc[ses[i],ses[j]] is np.NaN:
                    overlap.loc[ses[i],ses[j]] = 1
                else:
                    overlap.loc[ses[i],ses[j]] += 1
                if overlap.loc[ses[j],ses[i]] is np.NaN:
                    overlap.loc[ses[j],ses[i]] = 1
                else:
                    overlap.loc[ses[j],ses[i]] += 1
# print(overlap)

chosen_ones = ['ACA','Hippocampus','MoCo','PL', 'VisualAreas']
sessions = [np.unique(dat['new_area']) for dat in alldat]
overlap  = pd.DataFrame(index=chosen_ones, columns=chosen_ones)
for ses in sessions:
    for i in range(len(ses)-1):
        for j in range(i+1,len(ses)):
            if ses[i] in chosen_ones and ses[j] in chosen_ones:
                if overlap.loc[ses[i],ses[j]] is np.NaN:
                    overlap.loc[ses[i],ses[j]] = 1
                else:
                    overlap.loc[ses[i],ses[j]] += 1
                if overlap.loc[ses[j],ses[i]] is np.NaN:
                    overlap.loc[ses[j],ses[i]] = 1
                else:
                    overlap.loc[ses[j],ses[i]] += 1
# print(overlap)

# Making sure that the regions with all the chosen areas don't have other regions that are present in all
# Uncomment the code below for sanity check 
# chosen_ones = ['ACA','Hippocampus','MoCo','PL', 'VisualAreas']
# test = []
# for session in alldat:
#     session_areas = session['new_area']
#     if (set(np.intersect1d(chosen_ones,np.unique(session_areas))) == set(chosen_ones)):
#         test.append(session)
# test_areas = [np.unique(x['new_area']) for x in test]
# all_areas = np.unique(np.concatenate((test_areas)))
# t_overlap  = pd.DataFrame(index=all_areas, columns=all_areas)

# for i in range(len(test_areas)):
#     for j in range(len(test_areas[i])-1):
#         for k in range(i+1, len(test_areas[i])):
#             if t_overlap.loc[test_areas[i][j],test_areas[i][k]] is np.NaN:
#                 t_overlap.loc[test_areas[i][j],test_areas[i][k]] = 1
#             else:
#                 t_overlap.loc[test_areas[i][j],test_areas[i][k]] += 1
#             if t_overlap.loc[test_areas[i][j],test_areas[i][k]] is np.NaN:
#                 t_overlap.loc[test_areas[i][j],test_areas[i][k]] = 1
#             else:
#                 t_overlap.loc[test_areas[i][j],test_areas[i][k]] += 1

# Next we want to restrict the dataset to the sessions which have the 5 above areas, and only the neurons which belong to these areas
chosen_ones = ['ACA','Hippocampus','MoCo','PL', 'VisualAreas']
copy_as_is = ['bin_size', 'stim_onset','mouse_name','date_exp','wheel','pupil','response', \
              'response_time','contrast_right','contrast_left','feedback_time','feedback_type',\
              'gocue', 'licks','face', 'reaction_time', 'ccf_axes', 'prev_reward']
not_needed = ['waveform_w','waveform_u','brain_area_lfp','trough_to_peak','active_trials', \
              'contrast_left_passive','contrast_right_passive','spks_passive','lfp_passive',\
              'pupil_passive','wheel_passive', 'licks_passive', 'face_passive', 'cellid_orig' ]
restricted_dat = []
for session in alldat:
    session_areas = session['new_area']
    # Only if all the chosen_areas are in this session
    res_session = {}
    if (set(np.intersect1d(chosen_ones,np.unique(session_areas))) == set(chosen_ones)):
        valid = []
        for idx in range(len(session['new_area'])):
            if session['new_area'][idx] in chosen_ones:
                valid.append(idx)
        for key in session:
            if key not in not_needed:
                if key in copy_as_is:
                    res_session[key] = session[key]
                else:
                    #debug
                    if key == 'spks':
                        res_session[key] = np.take(session[key],valid, axis=0)
                    else:
                        res_session[key] = np.take(session[key],valid)
        restricted_dat.append(res_session)

# print("\n Now let's look at the  number of neurons in the original areas\n")
for dat in restricted_dat:
    this_dict = {}
    for x in np.unique(dat['brain_area']):
        this_dict[x] = 0
    for x in dat['brain_area']:
        this_dict[x] += 1
    # print(this_dict)

# We decided to separate Hippocampus back into CA1 and DG since they have enough
# neurons on their own, and the MoCO is just MOs. The Visual areas stay combined

for dat in restricted_dat:
    old_labels = dat['new_area']
    og_labels = dat['brain_area']
    for i in range(len(old_labels)):
        if old_labels[i] == 'Hippocampus' or old_labels[i] =='MoCo':
            old_labels[i] = og_labels[i]
    dat['new_area'] = old_labels

for dat in restricted_dat:
    this_dict = {}
    for x in np.unique(dat['new_area']):
        this_dict[x] = 0
    for x in dat['new_area']:
        this_dict[x] += 1
    # print(this_dict)

# Uncomment the code below for sanity check 
# for x in restricted_dat:
#     print(len(x['new_area']))
# for key in restricted_dat[0]:
#     try:
#         print(key, ' : ', restricted_dat[0][key].shape)
#     except:
#         print(key, restricted_dat[0][key])

# Reshape spikes from neurons x trials x timebins to trials x neurons x timebins

for dat in restricted_dat:
    dat['re_spks'] = np.swapaxes(dat['spks'], 0,1)

# Uncomment the code below for sanity check    
# for x in restricted_dat:
#     print(len(x['new_area']))
# for key in restricted_dat[0]:
#     try:
#         print(key, ' : ', restricted_dat[0][key].shape)
#     except:
#         print(key, restricted_dat[0][key])

# for dat in restricted_dat:
#     print(dat['gocue'].min(), dat['gocue'].max())

# We decided to use a time window of 400 msec before the go-cue onset
for dat in restricted_dat:
    sum_spikes = []
    for trial_num in range(len(dat['gocue'])):
        this_gocue = dat['gocue'][trial_num]
        # timebins = np.arange(0,2.5,0.01) #Incorrect!
        timebins = np.arange(-0.5,2,0.01)
        idx = np.argmin(np.abs(timebins - this_gocue))
        spk_count = np.sum(dat['re_spks'][trial_num][:,idx-40:idx], axis = 1)
        sum_spikes.append(spk_count)
    # print(np.array(sum_spikes).shape)
    dat['sum_spikes'] = np.array(sum_spikes)

# Uncomment the code below for sanity check 
# for x in restricted_dat:
#     print(len(x['new_area']))
# for key in restricted_dat[0]:
#     try:
#         print(key, ' : ', restricted_dat[0][key].shape)
#     except:
#         print(key, restricted_dat[0][key])